In [1]:
from src import train_dk, store_dk_model, load_dk_model, load_data_preprocess
from typing import Any, Tuple
import gpytorch
import torch

# parameters & settings
CSV_NO_817_URL = 'https://drive.google.com/uc?export=download&id=19IgGl230AbBuaH8q9wOxZ2R4OtRfHb7c'
X_attr=['T_cell', 'time', 'wavelength','R_0']
Y_attr='RT'
train_iter=500
lr=1e-2
verbose=True
gp_type='dk'
noise_constraint = gpytorch.constraints.Interval(1e-2, .1)
output_scale_constraint=None
kiss_gp=True
low_dim=True

In [2]:
# load data and preprocess
train_x, test_x, train_y, test_y = load_data_preprocess(csv_url=CSV_NO_817_URL, X_attr=X_attr, Y=Y_attr, test_size=0.25, random_state=11)

In [3]:
# Train the GPRegression model
model = train_dk(train_x=train_x, train_y=train_y, train_iter=train_iter, learning_rate=lr, verbose=verbose, gp_type=gp_type, noise_constraint=noise_constraint, output_scale_constraint=output_scale_constraint, low_dim=low_dim)

100%|██████████| 500/500 [05:29<00:00,  1.52it/s, Training Loss nll=-1.25, noise=0.0117, lengthscale=tensor([[2.3725]])]


In [4]:
# Store the model to a local path
model_path = "./model.pt"
store_dk_model(model, model_path)

In [10]:
# Load the model from the local path
# loaded_model = load_dk_model(model, model_path)

from src import GPRegressionModel, load_torch_model
loaded_model = GPRegressionModel(model.train_inputs[0], model.train_targets, model.likelihood, model.feature_extractor, low_dim=True)
loaded_model = load_torch_model(loaded_model, model_path)

In [13]:
# Use the loaded model to make predictions
loaded_model.eval()
loaded_model.likelihood.eval()
y_pred = loaded_model(test_x)


# Print the mean absolute error
print("Mean absolute error: %.2e" % torch.mean(torch.abs(test_y - y_pred.mean)))


Mean absolute error: 1.53e-02
